In [1]:
import os
import shutil
import PIL.Image

# first try used the below, second try will use even smaller images
# from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH
SMALLER_HEIGHT = 450 // 6
SMALLER_WIDTH = 600 // 6

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', '..', 'data2')
_dir = os.path.join(base_dir, 'archive', 'ISIC_2019_Training_Input')
# extended_dir = os.path.join(base_dir, 'extended')
extended_dir = os.path.join(base_dir, 'extended_75x100')

In [3]:
if not os.path.exists(extended_dir):
    PARTITION_SIZE = 5000
    files = os.listdir(_dir)
    jpg_files = [f for f in files if f.endswith('.jpg') or f.endswith('.jpeg')]
    current_partition = 0

    for idx in range(len(jpg_files)):
        if idx % PARTITION_SIZE == 0:
            current_partition += 1

        src = os.path.join(_dir, jpg_files[idx])
        partition_dir = os.path.join(extended_dir, f'partition_{current_partition}')

        if not os.path.exists(partition_dir):
            os.makedirs(partition_dir)

        dest = os.path.join(partition_dir, jpg_files[idx])

        shutil.copy(src, dest)

In [4]:
def resize_image(input_image_path, output_image_path, size):
    original_image = PIL.Image.open(input_image_path)
    width, height = original_image.size
    aspect_ratio = width / height

    if width > height:
        new_width = size[0]
        new_height = int(new_width / aspect_ratio)

        if new_height > size[1]:
            new_height = size[1]
            new_width = int(new_height * aspect_ratio)
    else:
        new_height = size[1]
        new_width = int(new_height * aspect_ratio)

        if new_width > size[0]:
            new_width = size[0]
            new_height = int(new_width / aspect_ratio)

    resized_img = original_image.resize((new_width, new_height))
    new_image = PIL.Image.new('RGB', size, 'black')

    new_image.paste(resized_img, ((size[0] - new_width) // 2,
                                  (size[1] - new_height) // 2))
    new_image.save(output_image_path)

In [5]:
dirs = os.listdir(extended_dir)

In [6]:
size = (SMALLER_WIDTH, SMALLER_HEIGHT)
# extended_and_resized = os.path.join(base_dir, 'extended_and_resized')
extended_and_resized = os.path.join(base_dir, 'extended_and_resized_75x100')

if not os.path.exists(extended_and_resized):
    os.makedirs(extended_and_resized)

for directory in dirs:
    full_path = os.path.join(extended_dir, directory)
    resized = os.path.join(full_path, 'resized')

    if not os.path.exists(resized):
        os.makedirs(resized)

    for file_name in os.listdir(full_path):
        if file_name.endswith('.jpg') or file_name.endswith('.png'):
            input_image_path = os.path.join(full_path, file_name)
            output_image_path = os.path.join(extended_and_resized, file_name)

            resize_image(input_image_path, output_image_path, size)